In [23]:
import requests
import json
import os
import pandas as pd
import numpy as np
from google.colab import drive
from datetime import date
from datetime import datetime


drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
s_qb = os.listdir('/content/drive/MyDrive/USPS project/QBitems/')
s_source = os.listdir('/content/drive/MyDrive/USPS project/Orders/')

print(s_source)
print(s_qb)


['Order_11042024.xlsx', 'Nov2024']
['Quickbooks 11182024.xlsx']


In [30]:
source_shopify = '/content/drive/MyDrive/USPS project/Orders/'
qb_output_path = '/content/drive/MyDrive/USPS project/QBitems/'

results     =    '/content/drive/MyDrive/USPS project/Output/NOV 2024/'


for i in s_source:
  if '.xlsx' in i:
    print(source_shopify+i)
    df_shopify = pd.read_excel(source_shopify+i)
  elif '.csv' in i:
    df_shopify = pd.read_csv(source_shopify+i)


for i in s_qb:
  if '.xlsx' in i:
    df_qb = pd.read_excel(qb_output_path+i)
  elif '.csv' in i:
    df_qb = pd.read_csv(qb_output_path+i)

print(df_qb.shape)
print(df_shopify.shape)

/content/drive/MyDrive/USPS project/Orders/Order_11042024.xlsx
(5525, 15)
(1197, 79)


In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
'''df_vcs1 = df_shopify['Lineitem sku'].value_counts().reset_index()
df_join_vcs = pd.merge(df_vcs1,df_vcs2, on = 'Lineitem sku', how='outer',suffixes=('_1', '_2'))
df_vcs2 = df_merged['Lineitem sku'].value_counts().reset_index()
df1final[df1final['SKU']=='C-3349666007921']
df_join_vcs[df_join_vcs['count_1']!=df_join_vcs['count_2']]
#print(df1final[['SKU','Product/Service Name']].shape)
df1vcssss = df1final[['SKU','Product/Service Name']].drop_duplicates().SKU.value_counts().reset_index()
df1vcssss[df1vcssss['count']==1]'''

"df_vcs1 = df_shopify['Lineitem sku'].value_counts().reset_index()\ndf_join_vcs = pd.merge(df_vcs1,df_vcs2, on = 'Lineitem sku', how='outer',suffixes=('_1', '_2'))\ndf_vcs2 = df_merged['Lineitem sku'].value_counts().reset_index()\ndf1final[df1final['SKU']=='C-3349666007921']\ndf_join_vcs[df_join_vcs['count_1']!=df_join_vcs['count_2']]\n#print(df1final[['SKU','Product/Service Name']].shape)\ndf1vcssss = df1final[['SKU','Product/Service Name']].drop_duplicates().SKU.value_counts().reset_index()\ndf1vcssss[df1vcssss['count']==1]"

In [33]:
def split(x):
  if type(x) == str:
    if x.count('-') ==2:
      a,b,c = x.split('-')
      return a+'-'+c
    else:
      return x
  else:
    return x

#df_shopify['Lineitem sku'][35:45]

In [34]:
#df_qb['SKU1'] = df_qb['SKU'].apply(lambda x : split(x))
df1final = df_qb[['Product/Service Name', 'SKU']]
df1final = df1final.dropna(subset=['SKU']) # dropping columns having same sku but different product names
df_shopify['Lineitem sku'] = df_shopify['Lineitem sku'].apply(lambda x : split(x))
df_shopify = df_shopify[['Created at', 'Lineitem quantity','Lineitem price','Lineitem sku','Notes','Payment Method']]
df_merged = pd.merge(df_shopify,df1final, left_on='Lineitem sku', right_on='SKU', how='left')

In [35]:
#Replace notes with proper customer name
def cleanup(x):
  if str(x) == 'nan':
    return x

  if 'Perfumora-Shein' in x:
    x = x.replace('Perfumora-Shein','Shein')
  elif 'Perfumora Amazon' in x:
    x = x.replace('Perfumora Amazon','Amazon')
  elif 'Wamart Perfumora' in x:
    x = x.replace('Wamart Perfumora','Walmart')
  elif 'Internet Fragrances - Amazon' in x:
    x = x.replace('Internet Fragrances - Amazon','Amazon_IFTG')
  elif 'perfumora-com' in x:
    x = x.replace('perfumora-com','Ebay')

  a = x.split(' ')
  return a[-1]

df_merged['Customer'] = df_merged['Notes'].apply(lambda x : cleanup(x))
df_merged.loc[df_merged['Payment Method'] == 'Stripe','Customer'] = 'Shopify - Customer'


# removing time in create at column
#df_merged['Created at'] = df_merged['Created at'].apply(lambda x : x.split()[0])

### to create invoice number
invoice_dict = {'Walmart' : 'W',
'Amazon' : 'A',
'Shein' : 'S',
'Amazon_IFTG' : 'I',
'Shopify - Customer' : 'P',
'Ebay' : 'E'
}
current_date_formatted = datetime.now().strftime("%Y-%m-%d").replace('-','')


df_merged['dt_inv'] = datetime.now().strftime("%m/%d/%Y")
df_merged['InvoiceNo'] = df_merged['Customer'].map(invoice_dict) + current_date_formatted
#df_merged['dt_inv'] = '10/15/2024'
#df_merged['InvoiceNo'] = df_merged['Customer'].map(invoice_dict) + '20241015'
print(df_merged['dt_inv'])
print(df_merged['InvoiceNo'])

0       11/24/2024
1       11/24/2024
2       11/24/2024
3       11/24/2024
4       11/24/2024
           ...    
1192    11/24/2024
1193    11/24/2024
1194    11/24/2024
1195    11/24/2024
1196    11/24/2024
Name: dt_inv, Length: 1197, dtype: object
0       A20241124
1       A20241124
2       A20241124
3       A20241124
4       W20241124
          ...    
1192    W20241124
1193    W20241124
1194    A20241124
1195    W20241124
1196    W20241124
Name: InvoiceNo, Length: 1197, dtype: object


In [36]:
df_final_quickbooks = pd.DataFrame(columns=['*InvoiceNo','*Customer','*InvoiceDate','*DueDate','Terms','Location','Memo','Item(Product/Service)','SKU number','ItemQuantity','ItemRate','ItemAmount','Taxable','TaxRate','Shipping address','Ship via','Shipping date','Tracking no','Shipping Charge','Service Date'])
df_final_quickbooks['*InvoiceNo'] = df_merged['InvoiceNo']
df_final_quickbooks['*Customer'] = df_merged['Customer']
df_final_quickbooks['*InvoiceDate'] = df_merged['dt_inv']
df_final_quickbooks['*DueDate'] = df_merged['dt_inv']
df_final_quickbooks['Item(Product/Service)']= df_merged['Product/Service Name']
df_final_quickbooks['SKU number'] = df_merged['Lineitem sku']
df_final_quickbooks['ItemQuantity']= df_merged['Lineitem quantity']
df_final_quickbooks['ItemRate']=df_merged['Lineitem price']
df_final_quickbooks['ItemAmount']= df_merged['Lineitem quantity'] * df_merged['Lineitem price']
df_final_quickbooks['Taxable']='N'
df_final_quickbooks['TaxRate']='5%'


In [37]:
for i in s_source:
  if '.xlsx' in i:
    sheet = results+ i


print(sheet)
df_final_quickbooks.to_excel(sheet,index=False)
#df_final_quickbooks['SKU number'][35:50]


/content/drive/MyDrive/USPS project/Output/NOV 2024/Order_11042024.xlsx
